In [ ]:
#LOAD DEPENDENCIES
import os
import logging
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from keras import applications
from keras.optimizers import Adam
from keras.models import load_model
from keras.models import Model, Input
from sklearn.utils import class_weight
from keras.applications.densenet import DenseNet121
from keras.applications.densenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers import Layer, ReLU, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Add, Concatenate, Dropout

#PREVENT ERROR UNCESSARY MESSAGES
tf.get_logger().setLevel(logging.ERROR)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
#LOAD THE DATA
train_data_dir = "data/train/"
validation_data_dir = "data/validation/"

# THE INPUT LAYER IS THE SAME AS IT WILL BE FUSED AS ONE LATER ON
img_rows, img_cols = 224, 224
input_shape = (img_rows,img_cols,3)
model_input = Input(shape=input_shape)
print("Data folders found!")
print("The Input size is set to ", model_input) 

In [ ]:
#DATA GENERATORS

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
                                
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows,img_cols),
        batch_size=batch_size,
        class_mode='categorical',
         classes=['0_Normal', '1_Covid19', '2_Pneumonia'])

validation_generator = val_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows,img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False,
         classes=['0_Normal', '1_Covid19', '2_Pneumonia'])

#CHECK  THE NUMBER OF SAMPLES
nb_train_samples = len(train_generator.filenames)
nb_validation_samples = len(validation_generator.filenames)

if nb_train_samples == 0:
    print("NO DATA TRAIN FOUND! Please check your train data path and folders!")
else:
    print("Train samples found!")
    
if nb_validation_samples == 0:
    print("NO DATA VALIDATION FOUND! Please check your validation data path and folders!")
    print("Check the data folders first!")
else:
    print("Validation samples found!")

#check the class indices
train_generator.class_indices
validation_generator.class_indices

#true labels
Y_test=validation_generator.classes
print(Y_test)

num_classes= len(train_generator.class_indices)

if nb_train_samples and nb_validation_samples > 0:
    print("Generators are set!")
    print("Check if dataset is complete and has no problems before proceeding.")

In [ ]:
#Weight each class

class_weights = class_weight.compute_class_weight(
               'balanced',
                np.unique(train_generator.classes), 
                train_generator.classes)
print(class_weights)

if class_weights != class_weights:
    print("Data imbalance detected!")

In [ ]:
# DenseNet121-A 

#TRANSFER LEARNING
def densenet_tiny_A_builder(model_input):
    densenet_tiny_A_builder = DenseNet121(weights='imagenet', include_top=False, input_tensor=model_input)
    
#Partial LAYER FREEZING
    for layer in densenet_tiny_A_builder.layers:
        layer.trainable = False 
        
    x = densenet_tiny_A_builder.layers[-354].output
    model = Model(inputs=densenet_tiny_A_builder.input, outputs=x, name='densenet-tiny-A')
    return model

#GENERATE THE MODEL
densenet_tiny_A = densenet_tiny_A_builder(model_input)

#PLOT THE MODEL STRUCTURE
densenet_tiny_A.summary()

In [ ]:
# DenseNet121-B

#TRANSFER LEARNING
def densenet_tiny_B_builder(model_input):
    densenet_tiny_B_builder = DenseNet121(weights='imagenet', include_top=False, input_tensor=model_input)
    
#RE-TRAINING ALL LAYERS (RE-NAMING LAYERS TO PREVENT OVERLAPS)
    for layer in densenet_tiny_B_builder.layers:
        layer.trainable = True
        layer.name = layer.name + str("_mirror")
        
    x = densenet_tiny_B_builder.layers[-354].output
    model = Model(inputs=densenet_tiny_B_builder.input, outputs=x, name='densenet_tiny-B')
    return model

#GENERATE THE MODEL
densenet_tiny_B = densenet_tiny_B_builder(model_input)

#PLOT THE MODEL STRUCTURE
densenet_tiny_B.summary()

In [ ]:
#PREPARE THE CONCATENATION OF THE PRE-TRAINED MODELS
densenet_tiny_A = densenet_tiny_A_builder(model_input)
densenet_tiny_B = densenet_tiny_B_builder(model_input)

print("DenseNet-Tiny-A and DenseNet-Tiny-B accomplished Pre-training and ready for concatenation")

In [ ]:
#CONCATENATE AS A SINGLE PIPELINE

models = [densenet_tiny_A, 
          densenet_tiny_B]

print("Concatenation success!")
print("Fused-DenseNet-Tiny ready to connect with its ending layers!")

Ensemble model definition is very straightforward. It uses the same input layer thas is shared between all previous models. 
In the top layer, the ensemble computes the average of three models' outputs (predictions) by using Average() layer. The ensemble is expected to have a lower error rate than any single model and better accuracy.

In [ ]:
#BUILD THE FUSED-DENSENET-TINY

def fused_densenet_tiny(models, model_input):
    outputs = [m.output for m in models]
    y = Add()(outputs)               
    y = GlobalAveragePooling2D()(y)
    y = Dense(512, activation='relu', use_bias=True)(y)
    y = Dropout(0.5)(y)
    prediction = Dense(num_classes,activation='softmax', name='Softmax_Classifier')(y)
    model = Model(model_input, prediction, name='fused_densenet_tiny')
    return model

#istantitate the ensemble model and report the summary
fused_densenet_tiny = fused_densenet_tiny(models,model_input)

print()
print()
print()
print("Fused-DenseNet-Tiny complete and ready for compilation and training!")
print()
print()
print()

fused_densenet_tiny.summary()

In [ ]:
#MODEL COMPILATION WITH HYPER-PARAMETERS, LOSS FUNCTIONS AND TRAINING!
import time

batch_size = 16

epochs = 25

start_time = time.time()

optimizer = Adam(lr=0.0001)

fused_densenet_tiny.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy']) 

reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=2,
                              verbose=1, mode='max', min_lr=0.000001)

callbacks = [reduce_lr]

history = fused_densenet_tiny.fit_generator(train_generator, steps_per_epoch=nb_train_samples // batch_size,
                                  epochs=epochs, validation_data=validation_generator,
                                  callbacks=callbacks, 
                                  validation_steps=nb_validation_samples // batch_size, verbose=1)

elapsed_time = time.time() - start_time
time.strftime("%H:%M:%S", time.gmtime(elapsed_time))

In [ ]:
#SAVE THE FUSED-DENSENET-TINY

fused_densenet_tiny.save('weights/fused_densenet_tiny.h5')

In [ ]:
#SAVE THE HISTORY FOR EVALUATION

from pickle import dump
dump(history, open('history/fused_densenet_tiny.pkl', 'wb'))